In [2]:
import numpy as np
import math 
from scipy.stats import norm
from scipy.stats import truncnorm
import scipy.stats as stats

In [3]:
import sys
import os

# Add the path to utils.py
sys.path.append(r"D:\Document\Serieux\Travail\python_work\cEBNM_torch\py")
sys.path.append(r"D:\Document\Serieux\Travail\python_work\cEBNM_torch\py\ebnm_solver")
sys.path.append(r"D:\Document\Serieux\Travail\python_work\cEBNM_torch\py\numerical_routine")

# Import utils.py directly
from utils import *
from numerical_routine import *
from distribution_operation import *
from posterior_computation import *

In [4]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])
scale = autoselect_scales_mix_norm ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1]))
location = 0* scale
n=betahat.shape[0]
p= scale.shape[0]
 
log_pi =  np.log( np.full( (n, p), 1/scale.shape[0]))
print(scale)
print(log_pi.shape)

convolved_logpdf_normal(betahat[1],sebetahat[1], location,scale)
print(get_data_loglik_normal (  betahat,sebetahat, location, scale))
#res = posterior_mean_exp(betahat, sebetahat, log_pi, scale)
#print(res.post_mean)
#print(res.post_mean2)

[0.         0.03827328 0.07654655 0.15309311 0.30618622 0.61237244
 1.22474487 2.44948974 4.89897949 9.79795897]
(5, 10)
[[ -1.41893853  -1.41893907  -1.41894705  -1.41907168  -1.42088747
   -1.44180176  -1.5770839   -1.96332218  -2.54837645  -3.21144866]
 [-12.5026478  -12.39380192 -12.07903895 -10.97389398  -8.11500906
   -4.34451202  -2.37716661  -2.15265225  -2.59406918  -3.22274394]
 [ -2.70837645  -2.7083952   -2.70845144  -2.70867638  -2.70957546
   -2.71316065  -2.72732222  -2.78109343  -2.95668542  -3.35402389]
 [ -8.91893853  -8.90796881  -8.87525773  -8.74731596  -8.27803033
   -6.89634722  -4.5770839   -3.03475075  -2.84837645  -3.28876825]
 [-13.41893853 -13.40138666 -13.34904414 -13.14426252 -12.39231604
  -10.16907449  -6.3770839   -3.67760789  -3.02837645  -3.33516   ]]


In [5]:

# Example Usage
data_loglik = get_data_loglik_normal (  betahat,sebetahat, location, scale)
assignment_loglik = log_pi 
    
res = apply_log_sum_exp(data_loglik, assignment_loglik)
print(res)

[[ -2.0667347   -2.06673523  -2.06674321  -2.06686784  -2.06868363
   -2.08959792  -2.22488006  -2.61111834  -3.19617261  -3.85924482]
 [-11.41455194 -11.30570607 -10.99094309  -9.88579812  -7.0269132
   -3.25641616  -1.28907075  -1.06455639  -1.50597332  -2.13464808]
 [ -2.22020012  -2.22021887  -2.22027512  -2.22050006  -2.22139914
   -2.22498433  -2.2391459   -2.2929171   -2.46850909  -2.86584757]
 [ -7.05746176  -7.04649204  -7.01378096  -6.88583918  -6.41655355
   -5.03487044  -2.71560713  -1.17327398  -0.98689967  -1.42729148]
 [-11.22103348 -11.2034816  -11.15113909 -10.94635747 -10.19441099
   -7.97116944  -4.17917885  -1.47970284  -0.83047139  -1.13725495]]


In [6]:
t_ind_Var =np.zeros((betahat.shape[0], scale.shape[0]))
 
for i in range(t_ind_Var.shape[0]):
    t_ind_Var[i , ]= np.concatenate(([0], 1/((1/sebetahat[i]**2)+ (1/scale[1:]**2)) ))#assume that first entry of scale is 0
    
 


temp=np.zeros((betahat.shape[0], scale.shape[0]))

for i in range(temp.shape[0]):
    temp[i,] = (t_ind_Var[i,]/(sebetahat[i]**2))*betahat[i]

print(temp)
post_mean  = np.sum( np.exp(res)* temp, axis=1)
post_mean2 = np.sum( np.exp(res)*( t_ind_Var+ temp**2), axis=1)
print(post_mean)
print(post_mean2)
print(post_mean2-post_mean **2)

[[0.00000000e+00 1.46270112e-03 5.82524272e-03 2.29007634e-02
  8.57142857e-02 2.72727273e-01 6.00000000e-01 8.57142857e-01
  9.60000000e-01 9.89690722e-01]
 [0.00000000e+00 1.81444297e-02 7.06547339e-02 2.55536627e-01
  7.38916256e-01 1.40186916e+00 1.80722892e+00 1.94805195e+00
  1.98675497e+00 1.99667221e+00]
 [0.00000000e+00 1.75770951e-04 7.02960244e-04 2.80986575e-03
  1.12079701e-02 4.43349754e-02 1.69811321e-01 5.80645161e-01
  1.46938776e+00 2.38016529e+00]
 [0.00000000e+00 5.85080449e-03 2.33009709e-02 9.16030534e-02
  3.42857143e-01 1.09090909e+00 2.40000000e+00 3.42857143e+00
  3.84000000e+00 3.95876289e+00]
 [0.00000000e+00 7.31350561e-03 2.91262136e-02 1.14503817e-01
  4.28571429e-01 1.36363636e+00 3.00000000e+00 4.28571429e+00
  4.80000000e+00 4.94845361e+00]]
[0.23635376 1.90130825 0.34311694 3.60843288 4.70131272]
[ 0.39779246  3.78405223  3.4020959  14.1843412  23.15589264]
[0.34192936 0.16907919 3.28436667 1.16355333 1.05355138]


The first posterior mean and second moment shoulld be 0.2363538 and 0.3977925

In [7]:
out= posterior_mean_norm(betahat, sebetahat, log_pi, scale)
print(out.post_mean)
print(out.post_mean2)

[0.23635376 1.90130825 0.34311694 3.60843288 4.70131272]
[ 0.39779246  3.78405223  3.4020959  14.1843412  23.15589264]


The first posterior mean and second moment shoulkd be 0.2363538 and 0.3977925

In [8]:
L= np.exp(data_loglik)
n, K = L.shape  # n: number of data points, K: number of components
pi =np.exp( - np.arange(0,K) )/ np.sum( np.exp( - np.arange(0,K) ))  # Initialize pi uniformly
print(pi.shape)
print(L.T.shape)
w = w = pi[:, np.newaxis] * L.T  # Now pi is reshaped to (10, 1) for broadcasting
 

print(w.sum(axis=1, keepdims=True))
w = w / w.sum(axis=1, keepdims=True)

(10,)
(10, 5)
[[1.95179041e-01]
 [7.18024847e-02]
 [2.64146951e-02]
 [9.71697530e-03]
 [3.57329107e-03]
 [1.34961162e-03]
 [5.90374744e-04]
 [2.25913377e-04]
 [6.60038750e-05]
 [1.46662010e-05]]


In [9]:
n, K = L.shape  # n: number of data points, K: number of components
print(K)
pi =np.exp( - np.arange(0,K) )/ np.sum( np.exp( - np.arange(0,K) ))  # Initialize pi uniformly
penalty= np.ones_like(pi)
penalty[0]=10
tol=1e-10
    # EM algorithm iterations
for iteration in range(20):
        # E-Step: calculate responsibilities (w_kj)
        w =  pi[:, np.newaxis] * L.T   # Element-wise multiplication (pi_k * l_kj)
        
        w = w / w.sum(axis=0, keepdims=True)  # Normalize by sum over k for each j

        # M-Step: update pi
        print( w.sum(axis=1) )
        print(penalty)
        n_k = w.sum(axis=1) + penalty - 1  # Apply penalty (S.2.8)
        pi_new = n_k / n_k.sum()  # Normalize to make sure sum(pi) = 1 (S.2.9)

        # Check for convergence
        if np.linalg.norm(pi_new - pi) < tol:
            print(f"Converged after {iteration} iterations.")
            break

        # Update pi
        pi = pi_new
print(pi)

10
[1.72677602 0.63749231 0.23710904 0.09181195 0.05115029 0.22287135
 0.66097632 0.82313638 0.43659911 0.11207723]
[10.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[1.6123663  0.09583589 0.0356601  0.01383433 0.00783523 0.05041024
 0.5127177  1.52890335 0.97549275 0.16694409]
[10.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[1.64135053e+00 1.48233723e-02 5.51687159e-03 2.14232155e-03
 1.22430039e-03 1.01582725e-02 2.74156512e-01 1.68822419e+00
 1.22429152e+00 1.38112106e-01]
[10.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[1.65956116e+00 2.31190199e-03 8.60587367e-04 3.34464955e-04
 1.92708353e-04 2.03417575e-03 1.40517316e-01 1.71114990e+00
 1.38084127e+00 1.02196517e-01]
[10.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[1.66762046e+00 3.61702866e-04 1.34664575e-04 5.23789064e-05
 3.04201762e-05 4.07678088e-04 7.15080173e-02 1.68816935e+00
 1.49911760e+00 7.25977243e-02]
[10.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[1.67175257e+00 5.66866030e-05 2.11084278e-05 8.21672393e-06
 4.80979152e-06 8.18344170e-05 3.63670